<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR_Unit1/blob/main/Lesson_HF_RL_Unit1%2Bbonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ССЫЛКА НА ОРИГИНАЛ:

https://colab.research.google.com/github/huggingface/deep-rl-class/blob/master/notebooks/bonus-unit1/bonus-unit1.ipynb

# Бонусный блок 1: Давайте научим собаку Хагги 🐶 приносить палку

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit2/thumbnail.png" alt="Bonus Unit 1Thumbnail">

В этой тетради мы закрепим то, что узнали в первом разделе, **научив собаку Хагги приносить палку, а затем играть с ней прямо в вашем браузере**

⬇️ Вот пример того, чего **вы достигнете в конце блока.** ⬇️ (запустите ▶ , чтобы увидеть)

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.mp4" type="video/mp4"></video>

### Окружающая среда 🎮

- Собака Хагги, окружающая среда, созданная [Thomas Simonini](https://twitter.com/ThomasSimonini) основанная на [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)

### Используемая библиотека 📚

- [MLAgents (Hugging Face version)](https://github.com/huggingface/ml-agents)

## Цели этого блокнота 🏆

В конце блокнота вы:

- Разберетесь с понятиями **пространство состояний, пространство действий и функция вознаграждения**, используемые для обучения Хагги.
- Научите **своего собственного Хагги** приносить палку.
- Получите возможность играть **со своим обученным Huggy прямо в вашем браузере**.



## Этот блокнот взят из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

В этом бесплатном курсе вы будете:

- 📖 Изучите глубокое обучение с подкреплением в **теории и практике**.
- 🧑‍💻 Научитесь **использовать известные библиотеки Deep RL**, такие как Stable Baselines3, RL Baselines3 Zoo, CleanRL and Sample Factory 2.0.
- 🤖 Обучайте **агентов в уникальных условиях** 

И еще проверьте учебную 📚 программу 👉 https://simoninithomas.github.io/deep-rl-course

The best way to keep in touch is to join our discord server to exchange with the community and with us 👉🏻 https://discord.gg/ydHrjt3WP5

## Предпосылки 🏗️

Прежде чем погрузиться в блокнот, вам нужно:

🔲 📚 **Развить понимание основ обучения с подкреплением** (MC, TD, Rewards hypothesis...) выполняя блок 1

🔲 📚 **Прочитать введение к Huggy**, выполнив бонусный блок 1

## Установите графический процессор 💪
- Чтобы **ускорить обучение агента, мы будем использовать графический процессор**. Чтобы сделать это, перейдите к `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1" width="40%">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2" width="30%">

## Клонируйте репозиторий и установите зависимости 🔽

- Нам нужно клонировать репозиторий, который **содержит экспериментальную версию библиотеки, которая позволяет вам отправлять вашего обученного агента в концентратор (Hub).**

In [ ]:
%%capture
# Клонирование этого конкретного репозитория (может занять 3 минуты)
!git clone https://github.com/huggingface/ml-agents/

In [ ]:
%%capture
# Зайдите в репозиторий и установите пакет (может занять 3 минуты)
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

## Загрузите и переместите zip-файл среды в `./trained-envs-executables/linux/`

- Исполняемый файл нашей среды находится в zip-файле.
- Нам нужно загрузить его и разместить на `./trained-envs-executables/linux/`

In [ ]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF" -O ./trained-envs-executables/linux/Huggy.zip && rm -rf /tmp/cookies.txt

Загрузите файл Huggy.zip с https://drive.google.com/uc?export=download&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF с помощью `wget`. Ознакомьтесь с полным решением для загрузки больших файлов из GDrive [здесь](https://bcrf.biochem.wisc.edu/2021/02/05/download-google-drive-files-using-wget/)

In [ ]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

Убедитесь, что ваш файл доступен

In [ ]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy

## Давайте кратко рассмотрим, как работает эта среда

### Пространство состояний: что "воспринимает" Хагги.

Хагги не "видит" свое окружение. Вместо этого мы предоставляем ему информацию об окружающей среде:

- Положение мишени (палки)
- Относительное положение Huggy и цели
- Ориентация его ног.

Учитывая всю эту информацию, Хагги **может решить, какие действия предпринять дальше, чтобы достичь своей цели**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.jpg" alt="Huggy" width="70%">


### Пространство действия: какие движения может делать Хагги
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-action.jpg" alt="Huggy action" width="70%">

**Суставные двигатели приводят в движение ноги Хагги**. Это означает, что для достижения цели он должен **научиться правильно вращать суставные двигатели каждой из своих ног, чтобы двигаться**.

### Функция вознаграждения

Функция вознаграждения разработана таким образом, чтобы **Хагги выполнил свою цель**: принес палку.

Помните, что одной из основ обучения с подкреплением является *гипотеза вознаграждения*: цель может быть описана как **максимизация ожидаемого совокупного вознаграждения**.

Наша цель состоит в том, чтобы Хагги **шел к палке, но не слишком сильно вращался**. Следовательно, наша функция вознаграждения должна транслировать эту цель.

Наша функция вознаграждения:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/reward.jpg" alt="Huggy reward function" width="70%">

- *Бонус за ориентацию*: мы **вознаграждаем агента за то, что он приблизился к цели**.
- *Штраф по времени*: штраф по фиксированному времени, назначаемый за каждое действие, чтобы **заставить его добраться до цели как можно быстрее**.
- *Штраф за вращение*: мы наказываем Хагги, если **он слишком много вращается и поворачивается слишком быстро**.
- *Награда за достижение цели*: мы вознаграждаем Хагги за **достижение цели**.